In [ ]:
#basic libraries
import pandas as pd
import numpy as np
import re, pprint

#Dimension Reduction Libraries
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF, PCA,KernelPCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans

#Visualizations
import pyLDAvis
import pyLDAvis.sklearn
import seaborn as sns
import matplotlib.pyplot as plt

#NLP Libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import spacy

#sklearn machine learning libraries
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler

%matplotlib inline

# 1. Reading In CSV File

In [2]:
#read in document

path= 'nlp_ml_ready_recipes.csv'

df = pd.read_csv(path, index_col = 0)

In [3]:
df.head()

,flavors,id,ingredients,recipeName,cuisine,piquant,meaty,bitter,sweet,sour,salty,nlp_ingredients
0,"{'piquant': 0.6666666666666666, 'meaty': 0.166...",Easy-Seafood-Paella-2077829,"['lobster_tails', 'water', 'olive_oil', 'onion...",Easy Seafood Paella,spanish,0.666667,0.166667,0.333333,0.166667,0.166667,0.166667,lobster_tail water olive_oil onion spanish_ric...
1,"{'piquant': 0.8333333333333334, 'meaty': 0.833...",Spanish-style-garlic-shrimp-298317,"['i_cant_believe_its_not_butter_spread', 'shri...",Spanish-style Garlic Shrimp,spanish,0.833333,0.833333,0.833333,0.166667,0.666667,0.833333,i_cant_believe_its_not_butter_spread shrimp sa...
2,"{'piquant': 0.16666666666666666, 'meaty': 0.16...",Classic-Paella-898818,"['olive_oil', 'chicken_breasts', 'chicken_stoc...",Classic Paella,spanish,0.166667,0.166667,0.333333,0.166667,0.666667,0.166667,olive_oil chicken_breast chicken_stock saffron...
3,"{'piquant': 0.16666666666666666, 'meaty': 0.16...",One-Pot-Spanish-Chicken-and-Rice-2237989,"['olive_oil', 'chicken_thighs', 'salt', 'peppe...",One Pot Spanish Chicken and Rice,spanish,0.166667,0.166667,0.166667,0.166667,0.833333,0.166667,olive_oil chicken_thighs salt pepper onion gar...
4,"{'piquant': 0.5, 'meaty': 0.16666666666666666,...",Portuguese-fish-stew-306395,"['plum_tomatoes', 'savoy_cabbage', 'olive_oil'...",Portuguese Fish Stew,spanish,0.500000,0.166667,0.166667,0.166667,0.500000,0.333333,plum_tomatoe savoy_cabbage olive_oil chizo_sau...


In [4]:
#pairwise distance function
def get_similiar_flavor_recipes(df,recipe,number):
    X = df.iloc[0:,2:-1].values
    dist_recipe = pairwise_distances(X, X[recipe].reshape(1,-1),metric ='euclidean')
    matches = np.argsort(dist_recipe.transpose()[0], axis = -1)
    return df.iloc[matches[:number]].drop(recipe,axis = 0)

In [5]:
#pairwise distance function
def get_similiar_ing_recipes(df,recipe,number):
    X = df.iloc[0:,2:-1].values
    dist_recipe = pairwise_distances(X, X[recipe].reshape(1,-1),metric ='euclidean')
    matches = np.argsort(dist_recipe.transpose()[0], axis = -1)
    return df.iloc[matches[:number]].drop(recipe,axis = 0)

# 2. Vectorizing of Text Data

In [6]:
stopwords = nltk.corpus.stopwords.words('english')


In [7]:
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df= 20,                        # minimum read occurences of a word 
                             stop_words= stopwords,             # remove stop words, using a custom list 
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}')  # num chars > 3
                             #max_features=500,             # max number of unique words
                            #ngram_range = (1,2))

data_vectorized = vectorizer.fit_transform(df['nlp_ingredients'])
vocab = vectorizer.get_feature_names()

In [8]:
#print(X)
df_vectors = pd.DataFrame(vectorizer.idf_, index=vectorizer.get_feature_names(), columns = ['Weight'])
 
# sort ascending

df_word_weights = df_vectors.sort_values(by = 'Weight').reset_index()
df_word_weights.head()

,index,Weight
0,salt,1.358510
1,pepper,1.363610
2,onion,1.450599
3,oil,1.452778
4,garlic,1.533094


In [9]:
vectorized_ingredients = pd.DataFrame(data_vectorized.toarray(), columns = vectorizer.get_feature_names())

vectorized_ingredients.head()

,active,adobo,agave,ale,allspice,almond,almonds,aminos,ancho,anchovy,...,wine,wing,wrapper,yeast,yogurt,yolk,yolks,yukon,zest,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.155878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#concating to main dataframe

df_ing = pd.concat([df[['recipeName','cuisine']], vectorized_ingredients], axis=1)

In [11]:
df_ing

,recipeName,cuisine,active,adobo,agave,ale,allspice,almond,almonds,aminos,...,wine,wing,wrapper,yeast,yogurt,yolk,yolks,yukon,zest,zucchini
0,Easy Seafood Paella,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Spanish-style Garlic Shrimp,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Classic Paella,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,One Pot Spanish Chicken and Rice,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,Portuguese Fish Stew,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.155878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
5,Portuguese Seafood Stew,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
6,Spanish Chicken with Chorizo and Potatoes,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
7,Mushroom Paella,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8,Spanish Chicken with Chorizo and Potatoes,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
9,Spanish-Style Chicken with Saffron Rice (Arroz...,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.138114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


# 3. Normalization of Flavor Vectors For Algos

In [12]:
#standardize flavors and ingredients for pairwise
X = df.iloc[0:,5:11]

scaler = StandardScaler()
df_normalize_features = scaler.fit_transform(X)

df_normalize_features = pd.DataFrame(df_normalize_features, columns=X.columns)

df_normalize_features = pd.concat([df[['recipeName','cuisine']], df_normalize_features], axis=1)
df_normalize_features.head()

,recipeName,cuisine,piquant,meaty,bitter,sweet,sour,salty
0,Easy Seafood Paella,spanish,1.342654,-0.858464,-0.305933,-0.36656,-1.030917,-1.157409
1,Spanish-style Garlic Shrimp,spanish,1.902339,1.772768,1.576510,-0.36656,0.797966,1.222303
2,Classic Paella,spanish,-0.336403,-0.858464,-0.305933,-0.36656,0.797966,-1.157409
3,One Pot Spanish Chicken and Rice,spanish,-0.336403,-0.858464,-0.933414,-0.36656,1.407593,-1.157409
4,Portuguese Fish Stew,spanish,0.782968,-0.858464,-0.933414,-0.36656,0.188338,-0.562481


In [13]:
get_similiar_flavor_recipes(df_normalize_features,0,5)

,recipeName,cuisine,piquant,meaty,bitter,sweet,sour,salty
130,Shrimp and Chorizo Rice,spanish,1.342654,-0.858464,-0.305933,-0.36656,-1.030917,0.032447
11315,Thai Stir-fried Chicken With Eggplant And Snak...,thai,1.342654,-0.858464,-0.305933,-0.36656,-1.030917,0.032447
7244,Mexican Chicken in Crockpot,mexican,1.342654,-0.858464,-0.305933,-0.36656,-1.030917,0.627375
140,Seafood Paella,spanish,1.342654,-0.858464,-0.305933,-0.36656,-1.030917,0.032447


In [14]:
get_similiar_ing_recipes(df_ing,0,-1)

,recipeName,cuisine,active,adobo,agave,ale,allspice,almond,almonds,aminos,...,wine,wing,wrapper,yeast,yogurt,yolk,yolks,yukon,zest,zucchini
288,Paella Bogavante,spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.127749,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
348,Easy Spanish Paella,spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5143,Suppeneintopf Gulasch Art,hungarian,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5146,Chili con carne Gulasch,hungarian,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
88,Spanish Garlic Prawns (Gambas al Ajillo),spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
535,Paella Roja,spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.145391,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
514,Saffron Shrimp Paella,spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
104,The Ultimate Paella,spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
695,Paella,spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
618,Mediterranean Paella (Vegetarian),spanish,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [15]:
X = df_normalize_features.iloc[0:,2:-1].values
X

array([[ 1.34265357, -0.85846414, -0.30593305, -0.36656036, -1.03091689],
       [ 1.90233924,  1.77276752,  1.57651005, -0.36656036,  0.79796551],
       [-0.33640346, -0.85846414, -0.30593305, -0.36656036,  0.79796551],
       ...,
       [ 0.22328222, -0.85846414,  1.57651005, -0.36656036,  1.40759297],
       [-0.33640346,  1.11495961,  0.94902902,  0.89088778,  0.79796551],
       [-0.89608914, -0.85846414,  1.57651005, -0.36656036,  0.18833804]])

In [16]:
recipe = 1
X[recipe].reshape(1,-1)

array([[ 1.90233924,  1.77276752,  1.57651005, -0.36656036,  0.79796551]])

In [17]:
#consider making this into a new data frame along with other ranking
dist = pd.DataFrame(pairwise_distances(X, X[recipe].reshape(1,-1),metric ='euclidean')) 

dist.head()

,0
0,3.758328
1,0.000000
2,3.934329
3,4.313550
4,3.853276


# 4. Composite Distance Recommender (Ingredients and Flavors)

In [18]:
#create a new function that takes in two datafames with ingredient and flavors info and the index of the dish
#returns a dataframe with the distances and the weighted average distance (composite)
#this is the first recommendation system

def top_dishes(recipe,ing_weight = 0.5,flavor_weight = 0.5):
    #define df_final
    df_final = df[['recipeName','cuisine']].copy()
    #generate flavor distance column
    X = df_normalize_features.iloc[0:,2:-1].values
    df_final['flavor_distance'] = pairwise_distances(X, X[recipe].reshape(1,-1),metric ='cosine')
    
    #generate ingredient distance column
    X2 = df_ing.iloc[0:,2:-1].values
    df_final['ing_distance'] = pairwise_distances(X2, X2[recipe].reshape(1,-1),metric ='cosine')
    
    #generate composite column
    
    df_final['composite_distance'] = (df_final['ing_distance']*ing_weight) + (df_final['flavor_distance']*flavor_weight)
    return df_final

In [19]:
#lets see how our recommendation system works
top_dishes(4035,ing_weight = 0.5,flavor_weight = 0.5).sort_values(by = 'composite_distance', ascending= True)

,recipeName,cuisine,flavor_distance,ing_distance,composite_distance
4035,New York Hot Dogs,american,1.110223e-16,0.000000,5.551115e-17
7752,Toad in the Hole,english,1.111739e-01,0.511120,3.111467e-01
4173,Carolina Pulled-Pork Sandwiches,american,1.337968e-02,0.731790,3.725847e-01
9376,Slow-Cooker Bratwurst with Sauerkraut and Apples,german,7.167985e-02,0.682132,3.769059e-01
10025,Curry Sausage German Style (Currywurst) from G...,german,2.183602e-01,0.584328,4.013442e-01
7391,Pork Enchiladas,mexican,1.337968e-02,0.891069,4.522241e-01
11154,Thai Chicken Curry Noodles,thai,1.110223e-16,0.908201,4.541003e-01
5081,Hungarian Veal Goulash,hungarian,9.196524e-02,0.817760,4.548626e-01
5006,Hungarian Goulash,hungarian,9.196524e-02,0.834003,4.629840e-01
10125,Slow-Cooked Spicy Portuguese Cacoila from Tast...,portuguese,1.039265e-01,0.831090,4.675081e-01


In [20]:
df[df['cuisine'] == 'irish']

,flavors,id,ingredients,recipeName,cuisine,piquant,meaty,bitter,sweet,sour,salty,nlp_ingredients
1431,"{'piquant': 0.16666666666666666, 'meaty': 0.16...",Karina_s-Irish-Beef-Stew-2250417,"['beef', 'sea_salt', 'mild_olive_oil', 'garlic...",Karina's Irish Beef Stew,irish,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,beef sea_salt mild_olive_oil garlic pearl_onio...
1432,"{'piquant': 0.0, 'meaty': 0.3333333333333333, ...",Corned-Beef-Hash-Simply-Recipes-42707,"['unsalted_butter', 'onions', 'cned_beef', 'po...",Corned Beef Hash,irish,0.000000,0.333333,0.833333,0.166667,0.833333,0.833333,unsalted_butter onion cned_beef potato salt pe...
1433,"{'piquant': 0.0, 'meaty': 0.16666666666666666,...",Slow-Cooker-Corned-Beef-and-Cabbage-1545600,"['beef_brisket', 'green_cabbage', 'potato', 'c...",Slow Cooker Corned Beef and Cabbage,irish,0.000000,0.166667,0.166667,0.166667,0.833333,0.166667,beef_brisket green_cabbage potato carrot onion...
1434,"{'piquant': 0.0, 'meaty': 0.16666666666666666,...",Crock-Pot-Corned-Beef-and-Cabbage-2262347,"['potato', 'onions', 'water', 'beef_brisket', ...",Crock Pot Corned Beef and Cabbage,irish,0.000000,0.166667,0.166667,0.166667,0.500000,0.166667,potato onion water beef_brisket beer cabbage
1435,"{'piquant': 0.8333333333333334, 'meaty': 0.166...",Corned-beef-poutine-with-guinness-gravy-351525,"['butter', 'onions', 'garlic', 'flour', 'guinn...",Corned Beef Poutine with Guinness Gravy,irish,0.833333,0.166667,0.833333,0.166667,0.833333,0.833333,butter onion garlic flour guinness_beer whole_...
1436,"{'piquant': 0.0, 'meaty': 0.6666666666666666, ...",Corned-Beef-_-Cabbage-1026704,"['beef_brisket', 'beer', 'green_cabbage', 'car...",Corned Beef & Cabbage,irish,0.000000,0.666667,0.166667,0.166667,0.833333,0.166667,beef_brisket beer green_cabbage carrot
1437,"{'piquant': 0.0, 'meaty': 0.16666666666666666,...",Crock-Pot-Corned-Beef-and-Cabbage-2063999,"['potato', 'onions', 'water', 'beef_brisket', ...",Crock Pot Corned Beef and Cabbage,irish,0.000000,0.166667,0.166667,0.166667,0.500000,0.166667,potato onion water beef_brisket beer cabbage
1438,"{'piquant': 0.0, 'meaty': 0.16666666666666666,...",Slow-Cooker-Corned-Beef-and-Cabbage-1030595,"['carrots', 'celery', 'onions', 'gold_potatoes...",Slow Cooker Corned Beef and Cabbage,irish,0.000000,0.166667,0.833333,0.166667,0.833333,0.833333,carrot celery onion gold_potatoe water cned_be...
1439,"{'piquant': 0.0, 'meaty': 0.16666666666666666,...",Slow-cooker-guinness-stew-345294,"['butter', 'beef_roast', 'salt', 'onion', 'cel...",Slow Cooker Guinness Stew,irish,0.000000,0.166667,0.166667,0.166667,0.666667,0.500000,butter beef_roast salt onion celery_ribs tomat...
1440,"{'piquant': 0.0, 'meaty': 0.16666666666666666,...",Crock-Pot-Corned-Beef-and-Cabbage-2007726,"['carrots', 'potato', 'onions', 'water', 'beef...",Crock Pot Corned Beef and Cabbage,irish,0.000000,0.166667,0.166667,0.166667,0.500000,0.166667,carrot potato onion water beef_brisket beer ca...


In [21]:
df.iloc[8312]['ingredients']


"['flank_steak', 'soy_sauce', 'dry_sherry', 'sugar', 'cn_starch', 'water', 'vegetable_oil', 'garlic_s', 'ginger_root', 'scallions', 'onions', 'green_pepper']"

In [22]:
df.iloc[10718]['ingredients']

"['chicken_breasts', 'bell_pepper', 'zucchini', 'onion', 'cherries', 'olive_oil', 'garlic', 'wine_vinegar', 'dried_egano', 'dried_rosemary', 'salt', 'pepper', 'kalamata', 'feta_cheese']"

# 5. Topic Modeling CorEX

In [23]:
#import CorEx libraries
from corextopic import corextopic as ct
from corextopic import vis_topic as vt 
import scipy.sparse as ss 

/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/networkx/classes/reportviews.py:95: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, Set, Iterable


In [24]:
#Exploring Ingredients to create anchors
df_vectors = pd.DataFrame(vectorizer.idf_, index=vectorizer.get_feature_names(), columns = ['Weight'])
 
# sort ascending
df_word_weights = df_vectors.sort_values(by = 'Weight').reset_index()
df_word_weights.sort_values(by = 'Weight', ascending = False).head(800)

,index,Weight
632,crusty,7.308532
623,angel,7.308532
617,poultry,7.308532
618,country,7.308532
619,clarified,7.308532
620,sushi,7.308532
621,wrapper,7.308532
622,chilli,7.308532
624,great,7.308532
625,nthern,7.308532


In [25]:
df_word_weights.to_csv('word_weights.csv')

In [26]:
df_word_weights.shape #837 distinct ingredients

(633, 2)

In [27]:
data_vectorized

<11533x633 sparse matrix of type '<class 'numpy.float64'>'
	with 208088 stored elements in Compressed Sparse Row format>

In [28]:
#data_vectorized 

vocab


['active',
 'adobo',
 'agave',
 'ale',
 'allspice',
 'almond',
 'almonds',
 'aminos',
 'ancho',
 'anchovy',
 'ange',
 'angel',
 'anise',
 'apple',
 'apples',
 'apricot',
 'arbio',
 'arrowroot',
 'artichoke',
 'arugula',
 'asian',
 'asparagus',
 'avocado',
 'baby',
 'back',
 'bacon',
 'baguette',
 'baking',
 'balsamic',
 'bamboo',
 'barbecue',
 'barley',
 'base',
 'basil',
 'basmati',
 'bass',
 'bay',
 'bbq',
 'bean',
 'beans',
 'beansprout',
 'beaten',
 'beef',
 'beer',
 'beet',
 'bell',
 'belly',
 'berrie',
 'blend',
 'blue',
 'boiling',
 'bok',
 'bone',
 'bonein',
 'boneless',
 'bouillon',
 'bouquet',
 'bourbon',
 'brat',
 'bread',
 'breadcrumb',
 'breadcrumbs',
 'breast',
 'breasts',
 'brisket',
 'broccoli',
 'broth',
 'bry',
 'bulb',
 'bun',
 'buns',
 'burger',
 'burgundy',
 'butt',
 'butter',
 'buttermilk',
 'butternut',
 'button',
 'cabbage',
 'cajun',
 'canned',
 'cannellini',
 'canola',
 'caper',
 'capsicum',
 'caraway',
 'cardamom',
 'carrot',
 'carrots',
 'cashew',
 'cashews'

In [29]:
anchors = [['shrimp','fish','lobster','haddock','catfish'],['beef','chicken','duck','chicken_breasts'],
           ['lettuce','arugula','spinach','greens','kale'],
           ['milk','butter','cheese','yogurt'],['pasta','linguine','grit','farro'], ['soup','stew','broth','chowder']]
#possible topics: seafood, grains, meat and poultry, fruits and vegetables, dairy heavy
#Salads, Seafood, Red Meats, Poultry, Fragrant, Grains
model = ct.Corex(n_hidden=6, seed=7) #so far 7 is best
model = model.fit(data_vectorized,words=vocab,anchors=anchors, # Pass the anchors in here
    anchor_strength=5)

/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.
/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.
/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.
/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning:

/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.
/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.
/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.
/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning:

In [30]:
model.tc #this is very low what is going on? It is not changing

0.6049051101442933

In [31]:
corex_topics = model.get_topics()
for n,topic in enumerate(corex_topics):
    topic_words,_ = zip(*topic)
    print('{}: '.format(n) + ', '.join(topic_words))

0: shrimp, fish, lobster, thai, mussel, saffron, coconut, chizo, clam, jumbo
1: beef, chicken, duck, lean, breast, brisket, wcestershire, thighs, potato, cilantro
2: spinach, lettuce, kale, arugula, greens, baby, romaine, iceberg, phyllo, hamburger
3: cheese, yogurt, parmesan, feta, cheddar, greek, mzarella, soy, shded, crumbles
4: pasta, linguine, penne, shell, flake, whipping, coarse, chard, thickcut, warm
5: soup, stew, meat, condensed, cream, mix, mushroom, meatball, broth, tabasco


In [32]:
model.get_topics(topic=0, n_words=20)

[('shrimp', 0.04177988796313691),
 ('fish', 0.03325854178324964),
 ('lobster', 0.0027943449527913044),
 ('thai', 0.0063438060970188795),
 ('mussel', 0.005693966333049182),
 ('saffron', 0.005624697547012227),
 ('coconut', 0.0041388699862327325),
 ('chizo', 0.0040084448764539805),
 ('clam', 0.003917865085791021),
 ('jumbo', 0.0034838287773686295),
 ('thread', 0.0029710038148790443),
 ('peanut', 0.002781411805140044),
 ('arbio', 0.002520844990431495),
 ('lemongrass', 0.0024247633176790948),
 ('palm', 0.002069331692015601),
 ('littleneck', 0.0018948695560290453),
 ('tamarind', 0.0018634586423089332),
 ('fillets', 0.0017445260407579054),
 ('threads', 0.001728408806558588),
 ('squid', 0.0016499281222453087)]

In [ ]:
plt.figure(figsize=(10,5))
plt.bar(range(model.tcs.shape[0]), model.tcs,color='#4e79a7', width=0.5)
plt.xlabel('Topic', fontsize=16)
plt.ylabel('Total Correlation (nats)', fontsize=16)

In [35]:
model.tcs

array([0.08997121, 0.15147171, 0.0570743 , 0.15964121, 0.06571977,
       0.08102691])

In [36]:
range(model.tcs.shape[0])

range(0, 6)

In [37]:
model.tcs.shape[0]

6

In [38]:
topic_df = pd.DataFrame(
    model.transform(data_vectorized), 
    columns=["topic_{}".format(i+1) for i in range(model.tcs.shape[0])]
).astype(float)
topic_df.index = df.index
df = pd.concat([df, topic_df], axis=1)

/anaconda3/lib/python3.7/site-packages/corextopic/corextopic.py:454: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  log_z = logsumexp(log_pygx_unnorm, axis=0)  # Essential to maintain precision.


In [39]:
df.head()

,flavors,id,ingredients,recipeName,cuisine,piquant,meaty,bitter,sweet,sour,salty,nlp_ingredients,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,"{'piquant': 0.6666666666666666, 'meaty': 0.166...",Easy-Seafood-Paella-2077829,"['lobster_tails', 'water', 'olive_oil', 'onion...",Easy Seafood Paella,spanish,0.666667,0.166667,0.333333,0.166667,0.166667,0.166667,lobster_tail water olive_oil onion spanish_ric...,1.0,0.0,0.0,0.0,0.0,0.0
1,"{'piquant': 0.8333333333333334, 'meaty': 0.833...",Spanish-style-garlic-shrimp-298317,"['i_cant_believe_its_not_butter_spread', 'shri...",Spanish-style Garlic Shrimp,spanish,0.833333,0.833333,0.833333,0.166667,0.666667,0.833333,i_cant_believe_its_not_butter_spread shrimp sa...,1.0,0.0,0.0,0.0,0.0,0.0
2,"{'piquant': 0.16666666666666666, 'meaty': 0.16...",Classic-Paella-898818,"['olive_oil', 'chicken_breasts', 'chicken_stoc...",Classic Paella,spanish,0.166667,0.166667,0.333333,0.166667,0.666667,0.166667,olive_oil chicken_breast chicken_stock saffron...,0.0,0.0,0.0,0.0,0.0,0.0
3,"{'piquant': 0.16666666666666666, 'meaty': 0.16...",One-Pot-Spanish-Chicken-and-Rice-2237989,"['olive_oil', 'chicken_thighs', 'salt', 'peppe...",One Pot Spanish Chicken and Rice,spanish,0.166667,0.166667,0.166667,0.166667,0.833333,0.166667,olive_oil chicken_thighs salt pepper onion gar...,0.0,0.0,0.0,0.0,0.0,0.0
4,"{'piquant': 0.5, 'meaty': 0.16666666666666666,...",Portuguese-fish-stew-306395,"['plum_tomatoes', 'savoy_cabbage', 'olive_oil'...",Portuguese Fish Stew,spanish,0.500000,0.166667,0.166667,0.166667,0.500000,0.333333,plum_tomatoe savoy_cabbage olive_oil chizo_sau...,1.0,0.0,0.0,0.0,0.0,0.0
